# 데이터 불러오기

In [1]:
!pip install dbfread
!pip install haversine
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dbfread import DBF
from haversine import haversine
from tqdm.notebook import tqdm
%matplotlib inline
import platform
platform.system()
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format = 'retina'

# 버스 정류장 승하차인원

출처:
    버스 시간대별 승하차 데이터:https://data.seoul.go.kr/dataList/OA-12913/S/1/datasetView.do
    버스 일일별 승하차 데이터:https://data.seoul.go.kr/dataList/OA-12912/A/1/datasetView.do;jsessionid=94B83FF22D72D1F4DCEB2322E53A6B03.new_portal-svr-21
    서울 지하철역 좌표: 서울시 지하철 주소를 위도경도로 바꿔주는 사이트(https://www.vworld.kr/dev/v4dv_geocoderguide2_s001.do)사용 후 csv파일로 저장
    

In [8]:
#동 위치정보, 버스정류장 승하차정보, 버스정류장 위치정보, 지하철역 위치정보
dong_info= DBF(r'./4개지역_행정동.DBF')
bus_info = pd.read_excel(r'./서울시버스정류소좌표데이터(2020.03.06).xlsx')
train_loc = pd.read_csv(r'./서울 지하철역 좌표.csv')
bus_num_1 = pd.read_csv(r'./2020-2019 버스 시간대별 승하차 데이터/1902버스_시간대별_승하차.csv')
bus_num_2 = pd.read_csv(r'./2020-2019 버스 시간대별 승하차 데이터/1903버스_시간대별_승하차.csv')
bus_num_3 = pd.read_csv(r'./2020-2019 버스 시간대별 승하차 데이터/1904버스_시간대별_승하차.csv')
bus_num_4 = pd.read_csv(r'./2020-2019 버스 시간대별 승하차 데이터/1905버스_시간대별_승하차.csv')
bus_num_5 = pd.read_csv(r'./2020-2019 버스 시간대별 승하차 데이터/2002버스_시간대별_승하차.csv')
bus_num_6 = pd.read_csv(r'./2020-2019 버스 시간대별 승하차 데이터/2003버스_시간대별_승하차.csv')
bus_num_7 = pd.read_csv(r'./2020-2019 버스 시간대별 승하차 데이터/2004버스_시간대별_승하차.csv')
bus_num_8 = pd.read_csv(r'./2020-2019 버스 시간대별 승하차 데이터/2005버스_시간대별_승하차.csv')

FileNotFoundError: [Errno 2] File ./2020-2019 버스 시간대별 승하차 데이터/1902버스_시간대별_승하차.csv does not exist: './2020-2019 버스 시간대별 승하차 데이터/1902버스_시간대별_승하차.csv'

In [13]:
dong_info = pd.DataFrame(dong_info)
dong_info

,STD_YYYY,HDONG_CD,HDONG_NM,SIDO_CD,SIDO_NM,SGNG_CD,SGNG_NM,AREA,LENGTH,X_COORD,Y_COORD
0,2020,2726067000,고산1동,27,대구광역시,27260,수성구,8881312.38,17757.54,128.69963062806,35.8117467030855
1,2020,2726068000,고산2동,27,대구광역시,27260,수성구,24523329.28,25763.81,128.677147073408,35.838758403345
2,2020,2726069000,고산3동,27,대구광역시,27260,수성구,5315712.45,11274.68,128.714043831276,35.8490479398919
3,2020,2726064000,두산동,27,대구광역시,27260,수성구,1966650.15,7777.10,128.620832776346,35.8285656536299
4,2020,2726055000,만촌1동,27,대구광역시,27260,수성구,2830625.94,8778.85,128.649513200936,35.8706818518603
...,...,...,...,...,...,...,...,...,...,...,...
64,2020,1114068000,중림동,11,서울특별시,11140,중구,482329.92,3500.86,126.966042028799,37.5573555328314
65,2020,1114064500,청구동,11,서울특별시,11140,중구,336949.37,2696.70,127.015201201613,37.5568631855271
66,2020,1114057000,필동,11,서울특별시,11140,중구,1173378.79,6575.62,126.992276757613,37.5568174037835
67,2020,1114067000,황학동,11,서울특별시,11140,중구,328678.23,2381.97,127.020846207973,37.5685359648242


In [14]:
bus_info

,표준ID,ARS-ID,정류장명,X좌표,Y좌표,비고
0,100000001,1001,종로2가사거리,126.987750,37.569765,NaN
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,NaN
2,100000003,1003,명륜3가.성대입구,126.998340,37.582671,NaN
3,100000004,1004,종로2가.삼일교,126.987613,37.568579,NaN
4,100000005,1005,혜화동로터리.여운형활동터,127.001744,37.586243,NaN
...,...,...,...,...,...,...
11275,124000334,25995,우성아파트,127.139339,37.550386,NaN
11276,124000333,25996,우성아파트,127.140046,37.550643,NaN
11277,124000332,25997,조일약국,127.123596,37.533630,NaN
11278,124000331,25998,성내시장,127.125497,37.536155,NaN


In [15]:
train_loc = pd.read_csv(r'./서울 지하철역 좌표.csv')
train_loc

,역명,상세주소,경도,위도,상태
0,서울,서울특별시 중구 봉래동2가 122 서울역(1호선),126.969563,37.558156,정좌표
1,시청,서울특별시 중구 정동 5-5 시청역(1호선),126.976919,37.565597,정좌표
2,종각,서울특별시 종로구 종로1가 54 종각지하철역사,126.981390,37.570222,정좌표
3,종로5가,서울특별시 종로구 종로5가 82-1 1호선 종로5가역,127.001925,37.571020,정좌표
4,동대문,서울특별시 종로구 창신동 492-1 1호선 동대문역(1호선),127.011233,37.571759,정좌표
...,...,...,...,...,...
273,부천종합운동장,경기도 부천시 원미구 춘의동 15-1 부천종합운동장역,126.797466,37.505639,실패
274,춘의,경기도 부천시 원미구 춘의동 145 춘의역,126.787091,37.503859,실패
275,신중동,경기도 부천시 원미구 중동 1140-6 신중동역,126.775952,37.503189,실패
276,부천시청,경기도 부천시 원미구 중동 1243 부천시청역,126.764025,37.504748,실패


# 필요한 컬럼만 뽑아오고 이름바꾸는 전처리

In [16]:
#동정보 전처리(동이름, 경도, 위도)
dong_loc = dong_info[['HDONG_NM', 'X_COORD', 'Y_COORD']]
dong_loc.columns=['동', '경도', '위도']
dong_loc.set_index('동', drop=True, inplace=True)
dong_loc

,경도,위도
동,,
고산1동,128.69963062806,35.8117467030855
고산2동,128.677147073408,35.838758403345
고산3동,128.714043831276,35.8490479398919
두산동,128.620832776346,35.8285656536299
만촌1동,128.649513200936,35.8706818518603
...,...,...
중림동,126.966042028799,37.5573555328314
청구동,127.015201201613,37.5568631855271
필동,126.992276757613,37.5568174037835


In [17]:
#버스정류장 위치정보 전처리(역명, id, 경도, 위도)
bus_loc=bus_info[['정류장명','표준ID', 'X좌표', 'Y좌표']]
bus_loc.columns=['역명', 'ID', '경도', '위도']
bus_loc.set_index('ID', drop=True, inplace=True)
bus_loc

,역명,경도,위도
ID,,,
100000001,종로2가사거리,126.987750,37.569765
100000002,창경궁.서울대학교병원,126.996566,37.579183
100000003,명륜3가.성대입구,126.998340,37.582671
100000004,종로2가.삼일교,126.987613,37.568579
100000005,혜화동로터리.여운형활동터,127.001744,37.586243
...,...,...,...
124000334,우성아파트,127.139339,37.550386
124000333,우성아파트,127.140046,37.550643
124000332,조일약국,127.123596,37.533630


## 지하철역 이름이 같고 위치가 다른 경우가 있음
## 다른호선이라 위도경도가 다르게 나옴

### 처음것만 남기고 없애는 중복제거 필요

In [18]:
#지하철 위치 전처리(역명, 경도, 위도)
train_loc = train_loc[['역명', '경도', '위도']]
train_loc=train_loc.drop_duplicates(['역명'], keep = 'first')
train_loc.set_index('역명', drop=True, inplace=True)
train_loc

,경도,위도
역명,,
서울,126.969563,37.558156
시청,126.976919,37.565597
종각,126.981390,37.570222
종로5가,127.001925,37.571020
동대문,127.011233,37.571759
...,...,...
부천종합운동장,126.797466,37.505639
춘의,126.787091,37.503859
신중동,126.775952,37.503189


# 지하철 위치정보와 버스정류장 위치정보를 가지고 가장 가까운 역 라벨링 하기

In [20]:
close_station = {'지하철역' : [],
                '표준버스정류장ID' : []}

for b_station in tqdm(bus_loc.index):
    start = (float(bus_loc.loc[b_station]['위도']), float(bus_loc.loc[b_station]['경도']))
    distance_dic = {}
    for t_station in train_loc.index:
        goal = (train_loc.loc[t_station]['위도'].mean(), train_loc.loc[t_station]['경도'].mean())
        distance_dic[t_station] = float(haversine(start, goal))
    close_ = sorted((value, key) for key, value in distance_dic.items())[0][1]
    close_station['표준버스정류장ID'].append(b_station)
    close_station['지하철역'].append(close_)

In [21]:
station_mapping = pd.DataFrame(close_station)
station_mapping = station_mapping[['표준버스정류장ID', '지하철역']]
station_mapping

,표준버스정류장ID,지하철역
0,100000001,종로3가(탑골공원)
1,100000002,혜화
2,100000003,혜화
3,100000004,을지로3가
4,100000005,한성대입구(삼선교)
...,...,...
11275,124000334,명일
11276,124000333,명일
11277,124000332,강동구청
11278,124000331,천호(풍납토성)


In [101]:
station_mapping.to_csv("busID_train_station_mapping.csv")

In [22]:
pd.read_csv('bus_Train_station_mapping.csv')

,Unnamed: 0,역명,지하철역
0,0,종로2가사거리,종로3가(탑골공원)
1,1,창경궁.서울대학교병원,혜화
2,2,명륜3가.성대입구,혜화
3,3,종로2가.삼일교,을지로3가
4,4,혜화동로터리.여운형활동터,한성대입구(삼선교)
...,...,...,...
6999,6999,강일리버파크1단지정문,상일동
7000,7000,굽은다리역,굽은다리(강동구민회관앞)
7001,7001,천호우체국.로데오거리,천호(풍납토성)
7002,7002,조일약국,강동구청


In [23]:
pd.read_csv('busID_Train_station_mapping.csv')

,Unnamed: 0,표준버스정류장ID,지하철역
0,0,100000001,종로3가(탑골공원)
1,1,100000002,혜화
2,2,100000003,혜화
3,3,100000004,을지로3가
4,4,100000005,한성대입구(삼선교)
...,...,...,...
11275,11275,124000334,명일
11276,11276,124000333,명일
11277,11277,124000332,강동구청
11278,11278,124000331,천호(풍납토성)


#### id로 하면 좀 더 많이 살릴 수 있다

# 2020/5월을 위해 역이름으로도 해줘야 한다.

In [24]:
#버스정류장 위치정보 전처리(역명, id, 경도, 위도)
bus_loc=bus_info[['정류장명','표준ID', 'X좌표', 'Y좌표']]
bus_loc.columns=['역명', 'ID', '경도', '위도']
bus_loc=bus_loc.drop_duplicates(['역명'], keep = 'first')
bus_loc.set_index('역명', drop=True, inplace=True)
bus_loc

,ID,경도,위도
역명,,,
종로2가사거리,100000001,126.987750,37.569765
창경궁.서울대학교병원,100000002,126.996566,37.579183
명륜3가.성대입구,100000003,126.998340,37.582671
종로2가.삼일교,100000004,126.987613,37.568579
혜화동로터리.여운형활동터,100000005,127.001744,37.586243
...,...,...,...
강일리버파크1단지정문,124900057,127.172870,37.568097
굽은다리역,124000398,127.142663,37.544322
천호우체국.로데오거리,124000335,127.127274,37.540556


In [33]:
#지하철 위치 전처리(역명, 경도, 위도)
train_loc = pd.read_csv(r'./서울 지하철역 좌표.csv')
train_loc = train_loc[['역명', '경도', '위도']]
train_loc=train_loc.drop_duplicates(['역명'], keep = 'first')
train_loc.set_index('역명', drop=True, inplace=True)
train_loc

,경도,위도
역명,,
서울,126.969563,37.558156
시청,126.976919,37.565597
종각,126.981390,37.570222
종로5가,127.001925,37.571020
동대문,127.011233,37.571759
...,...,...
부천종합운동장,126.797466,37.505639
춘의,126.787091,37.503859
신중동,126.775952,37.503189


In [27]:
close_station = {'지하철역' : [],
                '역명' : []}

for b_station in tqdm(bus_loc.index):
    start = (float(bus_loc.loc[b_station]['위도']), float(bus_loc.loc[b_station]['경도']))
    distance_dic = {}
    for t_station in train_loc.index:
        goal = (train_loc.loc[t_station]['위도'].mean(), train_loc.loc[t_station]['경도'].mean())
        distance_dic[t_station] = float(haversine(start, goal))
    close_ = sorted((value, key) for key, value in distance_dic.items())[0][1]
    close_station['역명'].append(b_station)
    close_station['지하철역'].append(close_)

In [28]:
station_mapping_ = pd.DataFrame(close_station)
station_mapping_ = station_mapping_[['역명', '지하철역']]
station_mapping_

,역명,지하철역
0,종로2가사거리,종로3가(탑골공원)
1,창경궁.서울대학교병원,혜화
2,명륜3가.성대입구,혜화
3,종로2가.삼일교,을지로3가
4,혜화동로터리.여운형활동터,한성대입구(삼선교)
...,...,...
6999,강일리버파크1단지정문,상일동
7000,굽은다리역,굽은다리(강동구민회관앞)
7001,천호우체국.로데오거리,천호(풍납토성)
7002,조일약국,강동구청


In [131]:
station_mapping_.to_csv("bus_train_station_mapping.csv")

In [29]:
station_mapping_

,역명,지하철역
0,종로2가사거리,종로3가(탑골공원)
1,창경궁.서울대학교병원,혜화
2,명륜3가.성대입구,혜화
3,종로2가.삼일교,을지로3가
4,혜화동로터리.여운형활동터,한성대입구(삼선교)
...,...,...
6999,강일리버파크1단지정문,상일동
7000,굽은다리역,굽은다리(강동구민회관앞)
7001,천호우체국.로데오거리,천호(풍납토성)
7002,조일약국,강동구청


In [30]:
pd.read_csv('bus_train_station_mapping.csv')

,Unnamed: 0,역명,지하철역
0,0,종로2가사거리,종로3가(탑골공원)
1,1,창경궁.서울대학교병원,혜화
2,2,명륜3가.성대입구,혜화
3,3,종로2가.삼일교,을지로3가
4,4,혜화동로터리.여운형활동터,한성대입구(삼선교)
...,...,...,...
6999,6999,강일리버파크1단지정문,상일동
7000,7000,굽은다리역,굽은다리(강동구민회관앞)
7001,7001,천호우체국.로데오거리,천호(풍납토성)
7002,7002,조일약국,강동구청


### 버스ID와 지하철역 라벨링이 끝났으니, 월별 버스승하차 정보가 있는 데이터 프레임에 ID기준으로 지하철역을 라벨링 해준다.

In [31]:
bus_num_1

,표준버스정류장ID,역명,사용년월,00시승차총승객수,00시하차총승객수,1시승차총승객수,1시하차총승객수,2시승차총승객수,2시하차총승객수,3시승차총승객수,...,19시승차총승객수,19시하차총승객수,20시승차총승객수,20시하차총승객수,21시승차총승객수,21시하차총승객수,22시승차총승객수,22시하차총승객수,23시승차총승객수,23시하차총승객수
0,100000001,종로2가사거리,201902,181,166,351,337,109,153,339,...,540,895,361,480,532,327,489,388,297,178
1,100000002,창경궁.서울대학교병원,201902,36,65,2,3,0,0,0,...,3581,849,2835,473,1448,525,950,448,740,194
2,100000003,명륜3가.성대입구,201902,482,240,24,27,0,0,0,...,5619,2951,5582,2022,6713,1680,7625,1446,3362,765
3,100000004,종로2가.삼일교,201902,69,27,188,66,83,44,113,...,1519,800,1503,595,1570,573,1627,550,620,156
4,100000005,혜화동로터리.여운형활동터,201902,108,283,12,29,0,0,0,...,1699,3701,1402,2207,1563,1865,1607,1529,497,826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12837,999800001,김포교통(기점가상),201902,0,0,0,0,0,0,0,...,17,2,10,1,25,0,18,0,5,1
12838,999800002,김포교통(종점가상),201902,1,2,0,0,0,0,0,...,0,1,0,2,0,3,0,3,0,0
12839,999800003,구미동차고지(기점가상),201902,0,2,0,0,0,0,0,...,1,2,6,2,4,3,2,1,1,0
12840,999800004,구미동차고지(종점가상),201902,2,15,0,5,0,0,0,...,0,5,0,14,0,7,0,6,0,9


In [32]:
bus_num_8

,역명,사용년월,00시승차총승객수,00시하차총승객수,1시승차총승객수,1시하차총승객수,2시승차총승객수,2시하차총승객수,3시승차총승객수,3시하차총승객수,...,19시승차총승객수,19시하차총승객수,20시승차총승객수,20시하차총승객수,21시승차총승객수,21시하차총승객수,22시승차총승객수,22시하차총승객수,23시승차총승객수,23시하차총승객수
0,(구)단대동주민센터,202005,223,81,192,15,47,0,0,0,...,294,181,249,149,261,146,296,63,154,119
1,11단지,202005,0,0,0,0,0,0,0,9,...,53,93,27,72,25,46,24,0,0,0
2,123전자타운.2001아울렛,202005,34,0,0,0,0,0,0,0,...,1014,1164,534,881,347,492,283,295,207,18
3,12번지건영아파트,202005,164,0,0,0,0,0,0,0,...,1058,164,976,101,831,66,796,25,529,1
4,13단지아파트입구,202005,299,71,104,6,7,0,0,0,...,664,294,652,320,593,241,525,145,377,190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8049,힐스테이트서울숲리버,202005,1,0,0,0,0,0,0,0,...,419,140,285,91,272,82,237,43,180,0
8050,힐스테이트송파.송파한라비발디,202005,186,9,48,0,0,0,0,0,...,854,12,496,5,518,10,468,20,442,22
8051,힐스테이트입구,202005,1,0,0,0,0,0,0,0,...,70,100,32,91,11,149,21,70,17,10
8052,힐탑,202005,0,0,0,0,0,0,0,0,...,37,8,21,3,39,0,37,0,19,0


In [35]:
bus_train_num_1 = pd.merge(bus_num_1, station_mapping, how='inner', on='표준버스정류장ID')
bus_train_num_2 = pd.merge(bus_num_2, station_mapping, how='inner', on='표준버스정류장ID')
bus_train_num_3 = pd.merge(bus_num_3, station_mapping, how='inner', on='표준버스정류장ID')
bus_train_num_4 = pd.merge(bus_num_4, station_mapping, how='inner', on='표준버스정류장ID')
bus_train_num_5 = pd.merge(bus_num_5, station_mapping, how='inner', on='표준버스정류장ID')
bus_train_num_6 = pd.merge(bus_num_6, station_mapping, how='inner', on='표준버스정류장ID')
bus_train_num_7 = pd.merge(bus_num_7, station_mapping, how='inner', on='표준버스정류장ID')
bus_train_num_8 = pd.merge(bus_num_8, station_mapping_, how='inner', on='역명')#아이디 없어서 따로 했음
bus_train_num_8

,역명,사용년월,00시승차총승객수,00시하차총승객수,1시승차총승객수,1시하차총승객수,2시승차총승객수,2시하차총승객수,3시승차총승객수,3시하차총승객수,...,19시하차총승객수,20시승차총승객수,20시하차총승객수,21시승차총승객수,21시하차총승객수,22시승차총승객수,22시하차총승객수,23시승차총승객수,23시하차총승객수,지하철역
0,123전자타운.2001아울렛,202005,34,0,0,0,0,0,0,0,...,1164,534,881,347,492,283,295,207,18,양천구청
1,12번지건영아파트,202005,164,0,0,0,0,0,0,0,...,164,976,101,831,66,796,25,529,1,구파발
2,14단지상가,202005,96,0,0,0,0,0,0,0,...,192,708,158,662,87,684,78,390,7,마들
3,1호선구일역,202005,61,0,0,0,0,0,0,0,...,251,507,223,437,243,317,95,199,15,남구로
4,2001아울렛중계점,202005,8,0,0,0,0,0,0,0,...,78,123,87,60,51,95,7,33,3,중계
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6506,힐스테이트서울숲리버,202005,1,0,0,0,0,0,0,0,...,140,285,91,272,82,237,43,180,0,금호
6507,힐스테이트송파.송파한라비발디,202005,186,9,48,0,0,0,0,0,...,12,496,5,518,10,468,20,442,22,장지
6508,힐스테이트입구,202005,1,0,0,0,0,0,0,0,...,100,32,91,11,149,21,70,17,10,길음
6509,힐탑,202005,0,0,0,0,0,0,0,0,...,8,21,3,39,0,37,0,19,0,녹번


# 완성...

In [36]:
bus_train_num_1.to_csv('1902버스지하철매칭 월별 승하차 합계.csv')
bus_train_num_2.to_csv('1903버스지하철매칭 월별 승하차 합계.csv')
bus_train_num_3.to_csv('1904버스지하철매칭 월별 승하차 합계.csv')
bus_train_num_4.to_csv('1905버스지하철매칭 월별 승하차 합계.csv')
bus_train_num_5.to_csv('2002버스지하철매칭 월별 승하차 합계.csv')
bus_train_num_6.to_csv('2003버스지하철매칭 월별 승하차 합계.csv')
bus_train_num_7.to_csv('2004버스지하철매칭 월별 승하차 합계.csv')
bus_train_num_8.to_csv('2005버스지하철매칭 월별 승하차 합계.csv')


# 날짜별 버스 사용하기 위해 ARS라벨링

In [11]:
bus_info = pd.read_excel(r'./서울시버스정류소좌표데이터(2020.03.06).xlsx')
train_loc = pd.read_csv(r'./서울 지하철역 좌표.csv')
bus_date = pd.read_csv('./BUS_STATION_BOARDING_MONTH_202002.csv',encoding = 'cp949')

## 라벨링을 위한 기본 전처리

In [12]:
train_loc

,역명,상세주소,경도,위도,상태
0,서울,서울특별시 중구 봉래동2가 122 서울역(1호선),126.969563,37.558156,정좌표
1,시청,서울특별시 중구 정동 5-5 시청역(1호선),126.976919,37.565597,정좌표
2,종각,서울특별시 종로구 종로1가 54 종각지하철역사,126.981390,37.570222,정좌표
3,종로5가,서울특별시 종로구 종로5가 82-1 1호선 종로5가역,127.001925,37.571020,정좌표
4,동대문,서울특별시 종로구 창신동 492-1 1호선 동대문역(1호선),127.011233,37.571759,정좌표
...,...,...,...,...,...
273,부천종합운동장,경기도 부천시 원미구 춘의동 15-1 부천종합운동장역,126.797466,37.505639,실패
274,춘의,경기도 부천시 원미구 춘의동 145 춘의역,126.787091,37.503859,실패
275,신중동,경기도 부천시 원미구 중동 1140-6 신중동역,126.775952,37.503189,실패
276,부천시청,경기도 부천시 원미구 중동 1243 부천시청역,126.764025,37.504748,실패


In [13]:
#지하철 위치 전처리(역명, 경도, 위도)
train_loc = train_loc[['역명', '경도', '위도']]
train_loc=train_loc.drop_duplicates(['역명'], keep = 'first')
train_loc.set_index('역명', drop=True, inplace=True)
train_loc

,경도,위도
역명,,
서울,126.969563,37.558156
시청,126.976919,37.565597
종각,126.981390,37.570222
종로5가,127.001925,37.571020
동대문,127.011233,37.571759
...,...,...
부천종합운동장,126.797466,37.505639
춘의,126.787091,37.503859
신중동,126.775952,37.503189


In [14]:
bus_col = bus_info.columns
bus_col

Index(['표준ID', 'ARS-ID', '정류장명', 'X좌표', 'Y좌표', '비고'], dtype='object')

In [15]:
bus_info_need = bus_info[['ARS-ID', '정류장명', 'X좌표', 'Y좌표']]
bus_info_need.columns =  ['ARS', '역명', '경도', '위도']
bus_info_need=bus_info_need.drop_duplicates(['ARS'], keep = 'first')
bus_info_need.set_index('ARS', drop=True, inplace=True)
bus_info_need

,역명,경도,위도
ARS,,,
1001,종로2가사거리,126.987750,37.569765
1002,창경궁.서울대학교병원,126.996566,37.579183
1003,명륜3가.성대입구,126.998340,37.582671
1004,종로2가.삼일교,126.987613,37.568579
1005,혜화동로터리.여운형활동터,127.001744,37.586243
...,...,...,...
25995,우성아파트,127.139339,37.550386
25996,우성아파트,127.140046,37.550643
25997,조일약국,127.123596,37.533630


## 버스 일별 데이터 전처리

In [16]:
#버스 일별 데이터 확인
bus_date

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20200201,100,100번(하계동~용산구청),11428,한성여객종점,14,4,20200204
1,20200201,100,100번(하계동~용산구청),11387,노원평생학습관,7,0,20200204
2,20200201,100,100번(하계동~용산구청),11373,중계역,27,2,20200204
3,20200201,100,100번(하계동~용산구청),11381,중계목화아파트4단지,120,13,20200204
4,20200201,100,100번(하계동~용산구청),11311,인덕대학,172,123,20200204
...,...,...,...,...,...,...,...,...
1118042,20200229,1111,1111번(번동~성북동),08128,KT월곡지사,43,38,20200303
1118043,20200229,1111,1111번(번동~성북동),08130,종암SK아파트,41,73,20200303
1118044,20200229,1111,1111번(번동~성북동),08134,동덕여대앞,75,58,20200303
1118045,20200229,1111,1111번(번동~성북동),08153,종암동주민센터,134,188,20200303


In [17]:
#역 갯수 확인
len(bus_date['역명'].unique())

8065

In [18]:
bus_col = bus_date.columns

In [19]:
bus_col = list(bus_col)
bus_col

['사용일자', '노선번호', '노선명', '버스정류장ARS번호', '역명', '승차총승객수', '하차총승객수', '등록일자']

In [20]:
#ARS기준으로 묶어줘야 할듯
bus_date_need = bus_date[['사용일자', '버스정류장ARS번호', '역명', '승차총승객수', '하차총승객수']]
bus_date_need.columns =  ['사용일자', 'ARS', '역명', '승차총승객수', '하차총승객수']
bus_date_need

,사용일자,ARS,역명,승차총승객수,하차총승객수
0,20200201,11428,한성여객종점,14,4
1,20200201,11387,노원평생학습관,7,0
2,20200201,11373,중계역,27,2
3,20200201,11381,중계목화아파트4단지,120,13
4,20200201,11311,인덕대학,172,123
...,...,...,...,...,...
1118042,20200229,08128,KT월곡지사,43,38
1118043,20200229,08130,종암SK아파트,41,73
1118044,20200229,08134,동덕여대앞,75,58
1118045,20200229,08153,종암동주민센터,134,188


In [21]:
bus_date_need['ARS']

0          11428
1          11387
2          11373
3          11381
4          11311
           ...  
1118042    08128
1118043    08130
1118044    08134
1118045    08153
1118046    08151
Name: ARS, Length: 1118047, dtype: object

## 같은날짜, 같은 역의 승하차 총 승객수로 묶어주자. 그러려면 ARS만 가지고 묶고 더하면 사용일자가 뭉개지니까 사용일자, ARS, 역명 기준으로 groupby랑 sum해주기

In [22]:
bus_date_updown = bus_date_need.groupby(['사용일자', 'ARS']).sum()
bus_date_updown.index

MultiIndex([(20200201, '01001'),
            (20200201, '01002'),
            (20200201, '01003'),
            (20200201, '01004'),
            (20200201, '01005'),
            (20200201, '01006'),
            (20200201, '01007'),
            (20200201, '01008'),
            (20200201, '01009'),
            (20200201, '01010'),
            ...
            (20200229, '68262'),
            (20200229, '68263'),
            (20200229, '68264'),
            (20200229, '68268'),
            (20200229, '68270'),
            (20200229, '68271'),
            (20200229, '68290'),
            (20200229, '68351'),
            (20200229, '68352'),
            (20200229,     '~')],
           names=['사용일자', 'ARS'], length=368013)

## 우선, ARS와 지하철역 거리기준으로 매칭

In [23]:
bus_date_need

,사용일자,ARS,역명,승차총승객수,하차총승객수
0,20200201,11428,한성여객종점,14,4
1,20200201,11387,노원평생학습관,7,0
2,20200201,11373,중계역,27,2
3,20200201,11381,중계목화아파트4단지,120,13
4,20200201,11311,인덕대학,172,123
...,...,...,...,...,...
1118042,20200229,08128,KT월곡지사,43,38
1118043,20200229,08130,종암SK아파트,41,73
1118044,20200229,08134,동덕여대앞,75,58
1118045,20200229,08153,종암동주민센터,134,188


In [24]:
bus_info_need

,역명,경도,위도
ARS,,,
1001,종로2가사거리,126.987750,37.569765
1002,창경궁.서울대학교병원,126.996566,37.579183
1003,명륜3가.성대입구,126.998340,37.582671
1004,종로2가.삼일교,126.987613,37.568579
1005,혜화동로터리.여운형활동터,127.001744,37.586243
...,...,...,...
25995,우성아파트,127.139339,37.550386
25996,우성아파트,127.140046,37.550643
25997,조일약국,127.123596,37.533630


In [26]:
train_loc

,경도,위도
역명,,
서울,126.969563,37.558156
시청,126.976919,37.565597
종각,126.981390,37.570222
종로5가,127.001925,37.571020
동대문,127.011233,37.571759
...,...,...
부천종합운동장,126.797466,37.505639
춘의,126.787091,37.503859
신중동,126.775952,37.503189


In [90]:
close_station = {'지하철역' : [],
                'ARS' : []}

for b_station in tqdm(bus_info_need.index):
    start = (float(bus_info_need.loc[b_station]['위도']), float(bus_info_need.loc[b_station]['경도']))
    distance_dic = {}
    for t_station in train_loc.index:
        goal = (train_loc.loc[t_station]['위도'].mean(), train_loc.loc[t_station]['경도'].mean())
        distance_dic[t_station] = float(haversine(start, goal))
    close_ = sorted((value, key) for key, value in distance_dic.items())[0][1]
    close_station['ARS'].append(b_station)
    close_station['지하철역'].append(close_)

In [96]:
station_mapping = pd.DataFrame(close_station)
station_mapping = station_mapping[['ARS', '지하철역']]
station_mapping['ARS']

0         1001
1         1002
2         1003
3         1004
4         1005
         ...  
11274    25995
11275    25996
11276    25997
11277    25998
11278    25999
Name: ARS, Length: 11279, dtype: int64

In [92]:
station_mapping.to_csv("busARS_train_station_mapping.csv")

In [27]:
station_mapping = pd.read_csv('busARS_Train_station_mapping.csv', index_col=0)
station_mapping

,ARS,지하철역
0,1001,종로3가(탑골공원)
1,1002,혜화
2,1003,혜화
3,1004,을지로3가
4,1005,한성대입구(삼선교)
...,...,...
11274,25995,명일
11275,25996,명일
11276,25997,강동구청
11277,25998,천호(풍납토성)


In [29]:
bus_date_updown.columns

Index(['승차총승객수', '하차총승객수'], dtype='object')

In [40]:
ARS_list = []
for ars in list(station_mapping['ARS']):
    if len(ars) == 5

[1001,
 1002,
 1003,
 1004,
 1005,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1014,
 1015,
 1016,
 1017,
 1018,
 1019,
 1020,
 1021,
 1022,
 1023,
 1037,
 1041,
 1044,
 1101,
 1102,
 1103,
 1104,
 1105,
 1106,
 1107,
 1108,
 1109,
 1110,
 1111,
 1112,
 1113,
 1114,
 1115,
 1116,
 1117,
 1118,
 1120,
 1123,
 1124,
 1125,
 1126,
 1127,
 1128,
 1129,
 1130,
 1131,
 1132,
 1133,
 1134,
 1135,
 1136,
 1137,
 1138,
 1139,
 1140,
 1141,
 1142,
 1143,
 1144,
 1146,
 1148,
 1149,
 1150,
 1151,
 1152,
 1153,
 1154,
 1155,
 1156,
 1157,
 1158,
 1159,
 1160,
 1161,
 1162,
 1163,
 1164,
 1165,
 1167,
 1168,
 1169,
 1171,
 1172,
 1173,
 1174,
 1175,
 1176,
 1177,
 1198,
 1199,
 1200,
 1201,
 1202,
 1203,
 1204,
 1205,
 1212,
 1213,
 1214,
 1215,
 1217,
 1218,
 1219,
 1220,
 1221,
 1222,
 1223,
 1224,
 1225,
 1226,
 1227,
 1228,
 1229,
 1231,
 1235,
 1236,
 1237,
 1238,
 1239,
 1240,
 1243,
 1245,
 1246,
 1247,
 1249,
 1250,
 1251,
 1252,
 1253,
 1254,
 1256,
 1262,
 1267,
 1272,
 1277,

In [37]:
group = bus_date_need.groupby(['사용일자', 'ARS']).sum()
group

승차총승객수  하차총승객수
사용일자     ARS                  
20200201 01001     326     446
         01002    1063     859
         01003    3076    1839
         01004     491     440
         01005    1135    2253
...                ...     ...
20200229 68271       7      10
         68290      34       5
         68351      60      12
         68352      11      50
         ~        4095    3568

[368013 rows x 2 columns]

해결해야 하는것
1. ARS가 4자리로 되어있는걸 5자리로 만들어줘야 한다
2. 날짜/ARS로 하니까 인덱스가 겹쳐버리는데, 날짜로만 인덱스 하고 ARS는 그냥 나중에 붙이는 식으로 해야할 듯. 